In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(42)

In [ ]:
class ConvBn(nn.Module):
    def __init__(self, in_channels,out_channels, kernel_size, stride=1, padding = 1):
        super(ConvBn, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding = padding, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)

    def forward(self, x):
            x = self.conv(x)
            x = self.bn(x)
            return x

# Pointwise and Depthwise Convolution - Separable Convolution
class SepBn(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding = 1):
        super(SepBn, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size, stride, padding=padding, groups=in_channels, bias=False)
        self.pointwise = nn.Conv2d(in_channels, out_channels, 1, 1, 0, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        
    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        x = self.bn(x)
        return x

In [ ]:
class XceptionNet(nn.Module):
  def __init__(self):
    super(XceptionNet,self).__init__()

    # EntryFlow
    self.conv1 = ConvBn(3, 32, 3, stride=2, padding = 1)
    self.relu1 = nn.ReLU()
    self.conv2 = ConvBn(32, 64, 3, padding = 1)
    self.relu2 = nn.ReLU()

    self.sep_bn1 = SepBn(64, 128, 3, padding = 1)
    self.relu3 = nn.ReLU()
    self.sep_bn2 = SepBn(128, 128, 3)
    self.pool1 = nn.MaxPool2d(kernel_size = 3, stride = 2, padding  = 1)

    self.conv3 = ConvBn(64, 128, kernel_size=1, stride=2, padding = 0)
    #add
    self.relu4 = nn.ReLU()
    self.sep_bn3 = SepBn(128, 256, 3)
    self.relu5 = nn.ReLU()
    self.sep_bn4 = SepBn(256, 256, 3)
    self.pool2 = nn.MaxPool2d(3, stride=2, padding  = 1)

    self.conv4 = ConvBn(128, 256, 1, stride=2,  padding = 0)
    #add
    self.relu6 = nn.ReLU()
    self.sep_bn5 = SepBn(256, 728, 3)
    self.relu7 = nn.ReLU()
    self.sep_bn6 = SepBn(728, 728, 3)
    self.pool3 = nn.MaxPool2d(3, stride=2, padding  = 1)

    self.conv5 = ConvBn(256, 728, 1, stride=2, padding = 0)
    #add

    # Middle Flow
    self.relu8 = nn.ReLU()
    self.sep_bn7 = SepBn(728,728,3)
    self.relu9 = nn.ReLU()
    self.sep_bn8 = SepBn(728, 728, 3)
    self.relu10 = nn.ReLU()
    self.sep_bn9 = SepBn(728, 728, 3)

    # Exit Flow
    self.sep_bn10 = SepBn(728,728,3)
    self.sep_bn11 = SepBn(728, 1024, 3)     
    self.poolMaxExit = nn.MaxPool2d(3, stride=2, padding  = 1)
    self.convExit = ConvBn(728, 1024, kernel_size=1, stride=2, padding = 0)
    self.sep_bn12 = SepBn(1024, 1536, 3)
    self.sep_bn13 = SepBn(1536, 2048, 3)
    self.avgpoolExit = nn.AdaptiveAvgPool2d((1,1))
    self.linearExit = nn.Linear(in_features=2048, out_features=2)
    self.softmaxExit = nn.LogSoftmax(dim=1)
    


  def forward(self, x):
      # Entry Flow
      x = self.conv1(x)
      x = self.relu1(x)
      x = self.conv2(x)
      x = self.relu2(x)
      tensor = x


      x = self.sep_bn1(tensor)
      x = self.relu3(x)
      x = self.sep_bn2(x)
      x = self.pool1(x)

      tensor1 = self.conv3(tensor)

      x = torch.add(tensor1, x)
      x = self.relu4(x)
      x = self.sep_bn3(x)
      x = self.relu5(x)
      x = self.sep_bn4(x)
      x = self.pool2(x)

      tensor2 = self.conv4(tensor1)

      x = torch.add(tensor2, x)
      x = self.relu6(x)
      x = self.sep_bn5(x)
      x = self.relu7(x)
      x = self.sep_bn6(x)
      x = self.pool3(x)

      tensor3 = self.conv5(tensor2)

      input = torch.add(tensor3, x)

      # Middle Flow
      for i in range(8):
        x = self.relu8(input)
        x = self.sep_bn7(x)
        x = self.relu9(x)
        x = self.sep_bn8(x)
        x = self.relu10(x)
        x = self.sep_bn9(x)
        input2 = torch.add(input,x)

      
      # Exit Flow
      x = self.relu10(input2)
      x = self.sep_bn10(x)
      x = self.relu10(x)
      x = self.sep_bn11(x)
      x = self.poolMaxExit(x)

      tensor = self.convExit(input2)

      x = torch.add(tensor, x)
      x = self.sep_bn12(x)
      x = self.relu10(x)
      x = self.sep_bn13(x)
      x = self.relu10(x)
      x = self.avgpoolExit(x)
      x = x.view(x.size(0), -1)
      x = self.linearExit(x)
      x = self.softmaxExit(x)

      return x

In [ ]:
image = torch.rand(1,3,299,299)
# image

In [ ]:
model = XceptionNet()

In [ ]:
op = model(image)

In [ ]:
op.shape

torch.Size([1, 2])

In [ ]:
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)
criterion = nn.BCELoss()

running_loss = 0.0
for epoch in range(1):
  for i, data in enumerate(image,0):
    inputs,labels = data

    optimizer.zero_grad()

    outputs = model(inputs)

    loss = criterion(outputs,labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % 64 == 63:
      print(f"[{epoch+1},{i + 1:5d}] loss: {running_loss / 64:.3f}")

print("Finished Training")

ValueError: ignored

In [1]:
# Test Image
label = 1
img = 0
# op = model(img)
op = 0.8 + img

In [3]:
if (op>=0.8):
  pred = 'real'
else:
  pred = 'fake'

print(pred)

real
